In [1]:
!pip install transformers datasets accelerate scikit-learn

In [2]:
from datasets import load_dataset

dataset = load_dataset("ailsntua/QEvasion")

train_ds = dataset["train"]
test_ds = dataset["test"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
set(train_ds["evasion_label"])

{'Claims ignorance',
 'Clarification',
 'Declining to answer',
 'Deflection',
 'Dodging',
 'Explicit',
 'General',
 'Implicit',
 'Partial/half-answer'}

In [4]:
label_list = sorted(list(set(train_ds["evasion_label"])))
label2id = {label:i for i,label in enumerate(label_list)}
id2label = {i:label for label,i in label2id.items()}

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("roberta-large")
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
def preprocess_evasion(batch):
    texts = [
        "Question: " + q + " Answer: " + a
        for q, a in zip(batch["interview_question"], batch["interview_answer"])
    ]
    tokenized = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokenized["labels"] = [label2id[label] for label in batch["evasion_label"]]
    return tokenized

In [8]:
train_enc = train_ds.map(
    preprocess_evasion,
    batched=True,
    remove_columns=train_ds.column_names
)

Map:   0%|          | 0/3448 [00:00<?, ? examples/s]

In [9]:
def preprocess_test(batch):
    texts = [
        "Question: " + q + " Answer: " + a
        for q, a in zip(batch["interview_question"], batch["interview_answer"])
    ]
    tokenized = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=512
    )
    return tokenized

test_enc = test_ds.map(
    preprocess_test,
    batched=True,
    remove_columns=test_ds.column_names
)

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    accuracy = accuracy_score(labels, preds)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "macro_f1": f1,
    }

In [ ]:
"""

model_evasion = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(label_list)
)

training_args = TrainingArguments(
    output_dir="evasion_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    metric_for_best_model="macro_f1",
    fp16=True,
    logging_dir="./evasion_logs"
)

trainer = Trainer(
    model=model_evasion,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=train_enc,
    compute_metrics=compute_metrics
)

trainer.train()
"""

In [12]:
model_evasion = AutoModelForSequenceClassification.from_pretrained(
    "roberta-large",
    num_labels=len(label_list)
)

training_args_v2 = TrainingArguments(
    output_dir="evasion_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    metric_for_best_model="macro_f1",
    fp16=True,
    logging_dir="./evasion_logs"
)

trainer_v2 = Trainer(
    model=model_evasion,
    args=training_args_v2,
    train_dataset=train_enc,
    eval_dataset=train_enc,
    compute_metrics=compute_metrics
)

trainer_v2.train()

Loading weights:   0%|          | 0/389 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: roberta-large
Key                             | Status     | 
--------------------------------+------------+-
lm_head.dense.weight            | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.dense.bias           | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Macro F1
1,No log,1.870973,0.302204,0.076700,0.196900,0.084371
2,No log,1.584031,0.394432,0.287006,0.339899,0.265613
3,1.800810,1.397150,0.468968,0.440560,0.455691,0.393403
4,1.800810,1.228411,0.555394,0.575497,0.504845,0.499522
5,1.402278,1.068799,0.620650,0.589570,0.571500,0.568318
6,1.402278,0.999330,0.647332,0.590104,0.599610,0.588781


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=1296, training_loss=1.5023472397415727, metrics={'train_runtime': 2241.3618, 'train_samples_per_second': 9.23, 'train_steps_per_second': 0.578, 'total_flos': 1.9280252041150464e+16, 'train_loss': 1.5023472397415727, 'epoch': 6.0})

In [13]:
predictions = trainer_v2.predict(test_enc)
preds = predictions.predictions.argmax(axis=1)
pred_labels = [id2label[p] for p in preds]

In [14]:
import pandas as pd


df = pd.DataFrame({
    "index": test_ds["index"],
    "predicted_evasion": pred_labels
})
df.to_csv("evasion_predictions_v2.csv", index=False)
print("Predictions saved to evasion_predictions.csv")

Predictions saved to evasion_predictions.csv
